In [1]:
from __future__ import division

In [2]:
import numpy as np
import pickle
import random

np.random.seed(42)

from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

# Function Definition

In [3]:
# function that calculates auc, recall and dcg of the evaulation result

def calc_metrics(infilename, trainfilename, gamma=0.2, K=10):
    infile = open(infilename, 'rb')
    P = pickle.load(infile)
    infile.close()
    NUM_NEGATIVES = P["num_negatives"]
    _NUM_POSs = dict()
    
    for theuser in P["users"]:
        _NUM_POSs[theuser] = len(P["user_items"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(P["user_items"][theuser])[-300:]
        P["results"][theuser] = list(P["results"][theuser])[-300:]
    
    Zui = dict()
    Ni = dict()
    
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][0 - _NUM_POSs[theuser]:]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
                
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][0 - _NUM_POSs[theuser]:]
        pos_scores = P["results"][theuser][0 - _NUM_POSs[theuser]:]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
            
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][0 - _NUM_POSs[theuser]:]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator
    
    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }


# Model Serving

In [4]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [5]:
raw_data = dict()
raw_data['train_data'] = np.load("training_arr.npy")
raw_data['val_data'] = np.load("validation_arr.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
val_dataset = ImplicitDataset(raw_data['val_data'], raw_data['max_user'], raw_data['max_item'], name='Val')

In [6]:
cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(), 
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix="./yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





2023-12-27 10:48:26.386092: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-12-27 10:48:26.403904: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fcc020590b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-12-27 10:48:26.403927: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [7]:
cml_model.load("cml-yahoo")

INFO:tensorflow:Restoring parameters from bpr-yahoo


In [8]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 300
model_trainer._exclude_positives([train_dataset, val_dataset])
model_trainer._sample_negatives(seed=10)
model_trainer._eval_save_prefix = "cml-yahoo-val-new"
model_trainer._evaluate_partial(val_dataset)

[Subsampling negative items]


100%|██████████| 7336/7336 [00:05<00:00, 1407.88it/s]  


{'AUC': [0.9833333333333334,
  0.5936734693877551,
  0.9866666666666667,
  0.8466666666666667,
  0.8093333333333333,
  0.9233333333333333,
  0.5866666666666667,
  0.8966666666666668,
  0.8144444444444444,
  0.7260000000000001,
  0.9633333333333334,
  0.9461111111111111,
  0.945,
  0.6416666666666666,
  0.9650000000000001,
  0.9066666666666668,
  0.8577777777777778,
  0.9966666666666667,
  0.9733333333333334,
  0.47333333333333333,
  0.9333333333333333,
  0.9019696969696969,
  0.9633333333333334,
  0.96,
  0.965,
  0.9133333333333333,
  0.9606666666666668,
  0.505,
  0.7866666666666666,
  0.8888888888888888,
  0.9933333333333333,
  0.9433333333333334,
  0.7466666666666667,
  0.9066666666666666,
  0.5677777777777777,
  0.98,
  0.9466666666666667,
  0.8975000000000001,
  0.9666666666666667,
  0.89,
  0.8266666666666667,
  0.8616666666666666,
  1.0,
  0.98,
  0.9566666666666667,
  0.7866666666666667,
  0.9633333333333334,
  0.9633333333333334,
  0.7849999999999998,
  0.9158333333333333,
  

# Evalution test set under different gamma values

In [9]:
# Evaluation on test set

test_eval_file = "cml-yahoo-val-new_evaluate_partial.pickle"
trainfilename = "training_arr.npy"

for gamma in [1.5, 2.0, 2.5, 3.0]:
      print(test_eval_file +  " with gamma @" + str(gamma) + " :")
      print(calc_metrics(test_eval_file, trainfilename, gamma, 1))

bpr-yahoo-val-new_evaluate_partial.pickle with gamma @1.5 :
{'auc': 0.8500056592649866, 'recall': 0.08267394750596867}
bpr-yahoo-val-new_evaluate_partial.pickle with gamma @2.0 :
{'auc': 0.8488026610644958, 'recall': 0.08174635098767072}
bpr-yahoo-val-new_evaluate_partial.pickle with gamma @2.5 :
{'auc': 0.8479489061102483, 'recall': 0.08108362340102103}
bpr-yahoo-val-new_evaluate_partial.pickle with gamma @3.0 :
{'auc': 0.847328025205392, 'recall': 0.08060074939560526}
